In [ ]:
import cv2
import numpy as np
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# sort images and reads the image using OpenCV's imread() function and stores it in the arr variable and then Resizes the image to a shape of (256, 256) using bilinear interpolation and stores it in the arr2 variable and append processed images to x.
from PIL import Image
import os
image_names=os.listdir('/content/drive/MyDrive/hazeimage/hazy')
folder_path = '/content/drive/MyDrive/hazeimage/hazy'
image_names.sort()
x=[]
for image_file in image_names:
    image_path = os.path.join(folder_path, image_file)
    arr=cv2.imread(image_path)
    #arr=cv2.cvtColor(arr,cv2.COLOR_BGR2RGB)
    #print(arr.shape)
    arr2=cv2.resize(arr,(256,256),interpolation=cv2.INTER_LINEAR)

    x.append(arr2)

In [ ]:
from PIL import Image
import os
image_names=os.listdir('/content/drive/MyDrive/hazeimage/GT')
folder_path = '/content/drive/MyDrive/hazeimage/GT'
image_names.sort()
y=[]
for image_file in image_names:
    image_path = os.path.join(folder_path, image_file)
    arr=cv2.imread(image_path)
    #arr=cv2.cvtColor(arr,cv2.COLOR_BGR2RGB)
    arr2=cv2.resize(arr,(256,256),interpolation=cv2.INTER_LINEAR)
    y.append(arr2)

In [ ]:
x=np.array(x)/255
y=np.array(y)/255
#Converts the list x containing processed images into a NumPy array, and then divides each pixel value by 255. This operation scales the pixel values to the range [0, 1], which is a common normalization technique for image data.
#By normalizing the pixel values to the range [0, 1], you ensure that the input data is scaled appropriately for training machine learning models, potentially improving convergence and performance during training.

In [ ]:
x[0].shape
#(256 pixels height,weidth and 3 color channels (RGB))

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.5)
# xtrain: Contains the training samples of the input data.
# xtest: Contains the testing samples of the input data.
# ytrain: Contains the corresponding ground truth (GT) training labels.
# ytest: Contains the corresponding GT testing labels.

In [ ]:
xtrain=np.array(xtrain)
ytrain=np.array(ytrain)
xtest=np.array(xtest)
ytest=np.array(ytest)
#converting each list into array

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x[10])
#to display 11th element from the x array

In [ ]:
# from tensorflow.keras.layers import Conv2D,MaxPooling2D,Input,UpSampling2D, BatchNormalization
# from tensorflow.keras.models import Model

# Imports various layers used for building convolutional neural networks (CNNs) and other types of neural network architectures.
# Conv2D: Convolutional layer for 2D spatial convolution.
# MaxPooling2D: Max pooling operation for spatial data.
# Input: Input layer for defining input shape.
# UpSampling2D: UpSampling layer for 2D inputs.
# Dropout: Dropout layer for regularization.
# BatchNormalization: Batch normalization layer to improve the training speed and stability of neural networks.
# Conv2DTranspose: Transposed convolution layer for upsampling.

In [ ]:
# Loss Functions : Mean Squared Error (MSE) (commonly used loss function in regression tasks, including image reconstruction tasks like image dehazing)
# Optimization Algorithms : Adam (Adam adapts the learning rates for each parameter during training, which can lead to faster convergence and better performance compared to using a fixed learning rate.)
# Activation Function :  Relu (introduces non-linearity to the network, allowing it to learn complex relationships between input features and output predictions, ReLU is simple to compute and does not involve complex mathematical operations. addresses the vanishing gradient problem )
# Training Hyperparameters : Learning rate , batch size, number of epochs(1000)
# Number of layers:
#              Convolutional layers: 10 (including the repeated blocks)
#              Max pooling layers: 5
#              Upsampling layers: 5
#              Batch normalisation :10
#          Total number of layers = Convolutional layers + Max pooling layers + Upsampling layers
#                                                         = 10 + 5 + 5 + 10= 30


#encoder start

# inputs=Input(shape=(256,256,3))

# con1=Conv2D(32,5,activation='relu',padding='same')(inputs)
# #con1=BatchNormalization()(con1)
# maxp=MaxPooling2D(2,padding='same')(con1)
# bn=BatchNormalization()(maxp)

# con2=Conv2D(64,5,activation='relu',padding='same')(bn)
# #con2=BatchNormalization()(con2)
# maxp1=MaxPooling2D(2,padding='same')(con2)
# bn=BatchNormalization()(maxp1)

# con3=Conv2D(80,5,activation='relu',padding='same')(bn)
# #con3=BatchNormalization()(con3)
# maxp2=MaxPooling2D(2,padding='same')(con3)
# bn=BatchNormalization()(maxp2)

# con3=Conv2D(128,5,activation='relu',padding='same')(bn)
# #con3=BatchNormalization()(con3)
# maxp2=MaxPooling2D(2,padding='same')(con3)
# bn=BatchNormalization()(maxp2)

# con3=Conv2D(256,5,activation='relu',padding='same')(bn)
# #con3=BatchNormalization()(con3)
# maxp2=MaxPooling2D(2,padding='same')(con3)
# bn=BatchNormalization()(maxp2)

# #encoder end

# #decoder start

# con4=Conv2D(256,5,activation='relu',padding='same')(bn)
# con4=BatchNormalization()(con4)
# up=UpSampling2D(2)(con4)

# con4=Conv2D(128,5,activation='relu',padding='same')(up)
# con4=BatchNormalization()(con4)
# up=UpSampling2D(2)(con4)

# con4=Conv2D(80,5,activation='relu',padding='same')(up)
# con4=BatchNormalization()(con4)
# up=UpSampling2D(2)(con4)

# con5=Conv2D(64,5,activation='relu',padding='same')(up)
# con5=BatchNormalization()(con5)
# up1=UpSampling2D(2)(con5)

# con6=Conv2D(32,5,activation='relu',padding='same')(up1)
# con6=BatchNormalization()(con6)
# up2=UpSampling2D(2)(con6)

# out=Conv2D(3,4,activation='sigmoid',padding='same')(up2)

# #decoder end

# model=Model(inputs=inputs,outputs=out)
# model.summary()

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D
from tensorflow.keras.models import Sequential

# Define the model function
def create_model():
    model = Sequential()

    # Encoder part
    model.add(Conv2D(32, 5, activation='relu', padding='same', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D(2, padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(64, 5, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(80, 5, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, 5, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 5, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, padding='same'))
    model.add(BatchNormalization())

    # Decoder part

    model.add(Conv2D(256, 5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(UpSampling2D(2))

    model.add(Conv2D(128, 5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(UpSampling2D(2))

    model.add(Conv2D(80, 5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(UpSampling2D(2))

    model.add(Conv2D(64, 5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(UpSampling2D(2))

    model.add(Conv2D(32, 5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(UpSampling2D(2))
# Decoder end
    model.add(Conv2D(3, 4, activation='sigmoid', padding='same'))



    return model

# Create the model
model = create_model()

# Print model summary
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
histr=model.fit(x,y,epochs=1000,validation_data=[xtest,ytest],batch_size=5)
#55/5 ->11 batches

In [ ]:
# Change the model compilation to include the 'accuracy' metric
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model again
histr = model.fit(x, y, epochs=1000, validation_data=[xtest, ytest], batch_size=5)

# Access the training accuracy from the history object
training_accuracy = histr.history['accuracy']

# Access the validation accuracy from the history object
validation_accuracy = histr.history['val_accuracy']


In [ ]:
model.save('model_without_dropout.keras')

In [ ]:
# Plot the training and validation accuracy

import matplotlib.pyplot as plt
# Assuming you have already trained your model and stored the training history in histr
# Accessing training accuracy
training_accuracy = histr.history['accuracy']
# Accessing validation accuracy
validation_accuracy = histr.history['val_accuracy']
# Generating the epoch numbers for the x-axis
epochs = range(1, len(training_accuracy) + 1)
# Plotting the training and validation accuracy
plt.plot(epochs, training_accuracy, 'r', label='Training Accuracy')
plt.plot(epochs, validation_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
#Validation loss vs epochs
import matplotlib.pyplot as plt
plt.plot(histr.history['val_loss'])


In [ ]:
#training loss vs epochs
import matplotlib.pyplot as plt
plt.plot(histr.history['loss'])


In [ ]:
#to view an image before dehazing
arr=cv2.imread('/content/drive/MyDrive/globeindiasmog_lrg.jpg')
#arr=cv2.cvtColor(arr,cv2.COLOR_BGR2RGB)
arr2=cv2.resize(arr,(256,256),interpolation=cv2.INTER_LINEAR)
arr2=np.array([arr2])/255
plt.title('image before')
plt.imshow(arr2[0])

In [ ]:
#to view an image after dehazing
arr=cv2.imread('/content/drive/MyDrive/globeindiasmog_lrg.jpg')
#arr=cv2.cvtColor(arr,cv2.COLOR_BGR2RGB)
arr2=cv2.resize(arr,(256,256),interpolation=cv2.INTER_LINEAR)
arr2=np.array([arr2])/255
#arr2=arr2/255
prerd=model.predict(arr2)
import matplotlib.pyplot as plt
prerd=prerd
plt.title('image after dehazing')
plt.imshow(prerd[0])

In [ ]:
#Training accuracy
training_accuracy = histr.history['accuracy']

In [ ]:
np.mean(training_accuracy)

In [ ]:
#Validation accuracy
validation_accuracy = histr.history['val_accuracy']


In [ ]:
np.mean(validation_accuracy)